In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Objective

What happens to performance when we add the features from the original dataset?

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))

In [3]:
from transform_for_num_issues_pred import main as transform_dataset

In [4]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

from helper_functions import dummify_cols_and_baselines, make_alphas

In [5]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [28]:
df_transformed = transform_dataset(df_orig)
df_transformed.shape

../preprocessing/transform_for_num_issues_pred.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_subset['NUM_ISSUES'] = 1


(822, 27)

In [29]:
df_transformed.head(1).T

,0
tract_and_block_group,1004002
race_white,0.242399
race_black,0.514358
race_asian,0.035473
race_hispanic,0.0675676
race_other,0.140203
poverty_pop_below_poverty_level,0
earned_income_per_capita,34340
poverty_pop_w_public_assistance,0.0597484
poverty_pop_w_food_stamps,0.138365


## Choosing columns

In [53]:
cols_orig_dataset = ['NUM_ISSUES_PER_100_POP', 'tract_and_block_group']
cols_census = ['race_white',
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'poverty_pop_below_poverty_level',
     'earned_income_per_capita',
     'poverty_pop_w_public_assistance',
     'poverty_pop_w_food_stamps',
     'poverty_pop_w_ssi',
     'school',
     'school_std_dev',
     'housing',
     'housing_std_dev',
     'bedroom',
     'bedroom_std_dev',
     'value',
     'value_std_dev',
     'rent',
     'rent_std_dev',
     'income',
     'income_std_dev']

In [54]:
df_col_subset = df_transformed[cols_orig_dataset + cols_census]
df_col_subset.shape

(822, 24)

In [55]:
df = df_col_subset.dropna().drop_duplicates()
df.shape

(558, 24)

## Removing outliers

0303003 is City Hall, which is where issues are assigned to when they don't have a location. Ideally, I would find out which issues truly took place in that block group and filter accordingly.

The other Census block groups I chose from looking at a Tableau cloropleth map. They are places with few, if any people living there, like parks and in the financial distinct.

In [56]:
outliers = ('9818001', '9811003', '9807001', '0303003', '0701018')

In [57]:
df = df[~df.tract_and_block_group.isin(outliers)].drop('tract_and_block_group', axis=1)
df.shape

(553, 23)

In [14]:
df_transformed.NUM_ISSUES_PER_100_POP.describe()

count     822.000000
mean      180.259308
std       450.539669
min         1.973490
25%        57.854814
50%        78.370274
75%       112.740311
max      4255.555556
Name: NUM_ISSUES_PER_100_POP, dtype: float64

In [13]:
df.NUM_ISSUES_PER_100_POP.describe()

count    553.000000
mean      83.072447
std       43.303655
min        1.973490
25%       56.995101
50%       77.142857
75%      103.289474
max      309.056604
Name: NUM_ISSUES_PER_100_POP, dtype: float64

## Dummify

In [58]:
cols_to_dummify = df.dtypes[df.dtypes == object].index
cols_to_dummify

Index([u'school', u'housing'], dtype='object')

In [59]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify)

8_6th_grade is baseline 0 2
rent is baseline 1 2


In [60]:
df_dummified.shape

(553, 32)

In [13]:
# df_dummified = df_dummified.drop('race_other', axis=1)

## Running model

In [18]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import make_scorer


/Users/datascientist/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [61]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('NUM_ISSUES_PER_100_POP', axis=1), 
    df_dummified.NUM_ISSUES_PER_100_POP, 
    test_size=0.2, 
    random_state=500
)

In [20]:
pipe = make_pipeline(StandardScaler(), LassoCV())

In [ ]:
mse = make_scorer(mean_squared_error, greater_is_better=False)

In [64]:
params = {'lassocv__alphas': make_alphas(-5, 5)}
params = {'lassocv__alphas': [[0.3], [1], [3]]}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=10)
model.fit(X_train, y_train);

In [65]:
pd.DataFrame(model.cv_results_).T.iloc[:5]

,0,1,2
mean_fit_time,0.0380748,0.034686,0.0235577
mean_score_time,0.00136201,0.000966716,0.000679612
mean_test_score,0.263573,0.26791,0.242252
mean_train_score,0.357083,0.340672,0.297388
param_lassocv__alphas,[0.3],[1],[3]


In [71]:
model.best_params_

{'lassocv__alphas': [1]}

In [66]:
model.score(X_test, y_test)

0.27634757102679774

### How many coefs at 0?

In [76]:
(model.best_estimator_.steps[-1][-1].coef_ == 0).sum()

14

In [77]:
X_train.shape[1]

31

14 out of 31 went to 0.

### Interesting  coef values?

In [78]:
coef_values = pd.DataFrame({
    'name': X_train.columns,
    'coef': model.best_estimator_.steps[-1][-1].coef_
})

coef_values['abs_coef'] = pd.np.abs(coef_values.coef)

In [79]:
coef_values[coef_values.coef != 0].sort_values('abs_coef', ascending=False)

,coef,name,abs_coef
11,-6.500630,housing_std_dev,6.500630
17,-6.412194,rent_std_dev,6.412194
6,5.969255,earned_income_per_capita,5.969255
3,-5.016315,race_hispanic,5.016315
5,-5.008938,poverty_pop_below_poverty_level,5.008938
14,4.014796,value,4.014796
10,-3.483669,school_std_dev,3.483669
28,-3.406515,school_21_masters,3.406515
12,-3.347862,bedroom,3.347862
20,-3.224426,school_0_none,3.224426


## Conclusion

Our baseline model's performance seems pretty bad, with $R^2=0.28$.